In [ ]:
import json
import gc
import numpy as np

from utils.dataset import DetectionFolder

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches

from sklearn.cluster import *

import urllib
from io import BytesIO
from PIL import Image
import os
import fnmatch

In [ ]:
w = 620
h = 620

image = Image.new('RGB', (w, h), 'black')
pixels = image.load()


max_val = 0
all_labels = []
for file_name in os.listdir('../data/detect/labels/'):
    if fnmatch.fnmatch(file_name, '*.txt'):
        with open('../data/detect/labels/' + file_name, 'r') as labelfile:
            labellines = labelfile.readlines()
            labels = np.array([labelline.split(',') for labelline in labellines])
            labels = labels.astype(np.float)
            labels = [[label[2] - label[0], label[3] - label[1]] for label in labels]
            for label in labels:
                all_labels.append(label)
                pixels[int(label[0]), int(label[1])] = tuple([v + 1 for v in pixels[int(label[0]), int(label[1])]])
                max_val = max(max_val, pixels[int(label[0]), int(label[1])][0])

max_rate = 250 / max_val

for i in range(0, w):
    for j in range(0, h):
        if pixels[i, j][0] == 0:
            pixels[i, j] = tuple([255, 255, 255])
        else:
            pixels[i, j] = tuple([0 , 0, 255])
                      

plt.imshow(image)
plt.show()


In [ ]:
kmeans = KMeans(n_clusters = 8, init = 'random').fit(all_labels) 

In [ ]:
print(kmeans.cluster_centers_)

In [ ]:
clusters = kmeans.cluster_centers_
clusters = np.array([[int(c[0]), int(c[1]), int(c[0] ** 2 + c[1] ** 2)] for c in clusters])
sorted_index = np.argsort(clusters[:, 2])
print(clusters[sorted_index])

_, ax = plt.subplots(1, 1, figsize=(19, 19), dpi=32)
ax.imshow(image)
for center_it in range(0, len(clusters)):
    center = kmeans.cluster_centers_[center_it]
            
            
    bounding = patches.Rectangle((center[0], center[1]), 3, 3, 
                linewidth=1, edgecolor='red', facecolor='red')
    ax.add_patch(bounding)
plt.show()